In [149]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

<span style="color:green; font-size: 35px; font-weight: bold;"> 데이터프레임과 시리즈(Pandas) </span></br>


# 9절. 데이터 그룹화 및 집계

In [5]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
- cf.pivot_table

### 1)그룹화 방법1 (df.groupby(by=[그룹핑할열])

In [25]:
# 1) 그룹화 방법1
#     df.groupby(by=[그룹핑할열])[출력될 열들].기초통계함수()
#                              [출력될 열들]생략시 그룹핑할 열을 제외한 모든 열 적용
# iris_grouped = iris.groupby(iris.Species) (비추)
# iris_grouped = iris.groupby(['Species'])  (비추)
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(3).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
10,5.4,3.7,1.5,0.2,setosa
23,5.1,3.3,1.7,0.5,setosa
44,5.1,3.8,1.9,0.4,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
69,5.6,2.5,3.9,1.1,versicolor
75,6.6,3.0,4.4,1.4,versicolor
94,5.6,2.7,4.2,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
109,7.2,3.6,6.1,2.5,virginica
136,6.3,3.4,5.6,2.4,virginica
145,6.7,3.0,5.2,2.3,virginica


In [ ]:
# 1) 그룹화 방법1
#아래 두개 함수는 같은 것이지만 아래것을 더 많이 씀
#어그리게이션펑션
iris.groupby(by='Species').sum()[['Sepal.Length']] 
iris.groupby(by='Species')[['Sepal.Length']].sum()

In [27]:
iris_grouped_sum = iris.groupby('Species').median()
iris_grouped_sum.reset_index()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [36]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [32]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [38]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa')|
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


### 2)그룹화 방법2 (피벗테이블)

In [ ]:
# 2) 그룹화 방법2
#     df.pivot_table(index=[그룹핑할열],
#                   values=[함수적용해서 출력할 열들],
#                   aggfunc=함수(기본값:mean))

In [55]:
iris.groupby('Species')[['Petal.Length','Petal.Width']].mean()
iris.pivot_table(index='Species',
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [51]:
iris.groupby('Species').count() #결측치를 제외한 값의 갯수

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [52]:
iris.pivot_table(index='Species',
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [56]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [57]:
# iris로 다중열로 그룹화하기 위해 데이터 column 추가(num 필드)
iris.shape

(150, 5)

In [62]:
# [[0]*25] + [[1]*25] + [[2]*25] + [[3]*25]
import numpy as np
np.ravel([[i]*25 for i in range(6)]) #1차원 데이터로 

# [[i]*25 for i in range(6)] #2차원 [[]]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [79]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [80]:
iris.shape

(150, 6)

In [83]:
import random
# random.randint(50)
#iris.iloc[random.sample(range(149),3),0]
# iris.iloc[0,0] 

126    6.2
62     6.0
34     4.9
Name: Sepal.Length, dtype: float64

# 질문 

In [84]:
# 각 열의 임의의 데이터에 결측치를 할당
import random
print(random.sample(range(149),3))
for col in range(iris.shape[1]-2): #Species,num 제외(-2)
    iris.iloc[random.sample(range(149), col+1), col] = np.nan

# iris.iloc[random.sample(range(149),3),3] = np.nan

[136, 18, 129]


In [85]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [87]:
#그룹별 집계 (Species, num별 카운트)
iris.groupby(by=['Species','num'])[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              25           24
           3              24           24
virginica  4              25           25
           5              24           25

In [88]:
iris.pivot_table(index=['Species','num'],
                 values=['Petal.Length','Petal.Width'],
                 aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           24
           1              24           24
versicolor 2              25           24
           3              24           24
virginica  4              25           25
           5              24           25

In [92]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  50.000000   50.000000  49.000000
             mean    5.006000    5.936000   6.608163
             std     0.352490    0.516171   0.626111
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.300000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   48.000000  50.000000
             mean    3.428000    2.766667   2.974000
             std     0.379064    0.319796   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.500000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
Petal.Length count  49.000000   49.000000  49.000000
             mean    1.465306    4.265306   5.567347
             std     0.173867    0.473265   0.546728
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.400000   5.600000
             75%     1.600000    4.600000   5.900000
             max     1.900000    5.100000   6.900000
Petal.Width  count  48.000000   48.000000  50.000000
             mean    0.241667    1.320833   2.026000
             std     0.100707    0.199956   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [96]:
iris.drop('num',axis=1, inplace=True)

KeyError: "['num'] not found in axis"

In [ ]:
# 독립변수(넘파이 배열)
# 종속변수 Species
# Species를 나눠서 setosa=0,versicolor=1,virginica=2 (레이블 인코딩)

In [101]:
#레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder #LabelEncoder 클래스
le = LabelEncoder() #객체 생성
iris['taget'] = le.fit_transform(iris.Species) #iris의 Species 기준으로 변환
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,taget
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 ↔ 롱포맷
- 판다스 공식 문서의 melt : https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt
- Tidy Data(7page) : https://vita.had.co.nz/papers/tidy-data.pdf

In [107]:
# 와이드 포맷 데이터 
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality',
                                   package='datasets').data
airquality

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5
...,...,...,...,...,...,...
148,30.0,193.0,6.9,70,9,26
149,NaN,145.0,13.2,77,9,27
150,14.0,191.0,14.3,75,9,28
151,18.0,131.0,8.0,76,9,29


## 10.1 melt를 이용한 언피벗팅(와이드 포맷 -> 롱 포맷)

In [116]:
# 데이터 구조 변경 (와이드 포맷 -> 롱 포맷)
air_melted = airquality.melt(id_vars=['Month','Day'],
               var_name='변수명',
               value_name='값')
air_melted.iloc[::20]
air_melted.sort_values(by=['Month','Day'], inplace=True)

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [117]:
#melt 연습하기(와이드 포맷 데이터:부동산 -> 롱 포맷으로 변환)
%ls C:\ai_x\download\ShareData\부동산_250213

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\ai_x\download\ShareData\부동산_250213 디렉터리

2025-04-30  오후 02:44    <DIR>          .
2025-04-30  오후 02:44    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  160,907,317,248 바이트 남음


In [123]:
import pandas as pd

In [131]:
# 지역/ 연도월 / 평당분양가


In [128]:
df.columns

Index(['지역', '2013년12월', '2014년1월', '2014년2월', '2014년3월', '2014년4월', '2014년5월',
       '2014년6월', '2014년7월', '2014년8월', '2014년9월', '2014년10월', '2014년11월',
       '2014년12월', '2015년1월', '2015년2월', '2015년3월', '2015년4월', '2015년5월',
       '2015년6월', '2015년7월', '2015년8월'],
      dtype='object')

In [138]:
df = pd.read_csv(r'C:\ai_x\download\ShareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv',
                 encoding='cp949')

new_df = df.melt(id_vars='지역',
        var_name ='연도월',
        value_name='평당분양가')
new_df.sample()

,지역,연도월,평당분양가
215,충남,2014년12월,6989


## 10.2 pivot_table을 이용한 피벗팅(롱포맷->와이드포맷)

In [140]:
# 롱포맷-> 와이드포맷
#air_melted 
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
1,5,2,Ozone,36.0
2,5,3,Ozone,12.0
3,5,4,Ozone,18.0
4,5,5,Ozone,NaN


In [147]:
airquality2 = air_melted.pivot_table(index=['Month','Day'],
                                           columns='변수명',
                                           values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name=None
airquality2

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3
...,...,...,...,...,...,...
148,9,26,30.0,193.0,70.0,6.9
149,9,27,NaN,145.0,77.0,13.2
150,9,28,14.0,191.0,75.0,14.3
151,9,29,18.0,131.0,76.0,8.0


# 11절. 데이터프레임(시리즈)에 함수 적용
- apply(func, axis) : 축요소별(요소별) 함수 적용(데이터 프레임이나 시리즈에서 가능)
        처음부터 끝까지 적용 시키는것 
- applymap(func) : 각 요소별 함수 적용(데이터 프레임에서만 가능-2차원에서만 사용 가능)
        모든 요소에 다 적용 시키는 것
- map(func, list) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [157]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [171]:
def get_year(datestr):
    return int(datestr.split('-')[0]) #람다식 

In [173]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [174]:
get_year('1999-05-01')

1999

In [175]:
get_month = lambda datestr : int(datestr.split('-')[1])

In [176]:
get_month('1999-04-01')

4

In [179]:
member['Year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]),)

In [180]:
member

,Name,Age,Email,Address,Birth,Year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [ ]:
member['Month'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[1]),)

In [186]:
X = iris.iloc[:,:-2] 
#X의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.844966
Sepal.Width     3.060135
Petal.Length    3.765986
Petal.Width     1.207534
dtype: float64

# 질문

In [187]:
X.iloc[0]-avg

Sepal.Length   -0.744966
Sepal.Width     0.439865
Petal.Length   -2.365986
Petal.Width    -1.007534
dtype: float64

In [188]:
#X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리만 출력
X.apply(lambda x : round(x-avg, 2),axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.74,0.44,-2.37,-1.01
1,-0.94,-0.06,-2.37,-1.01
2,-1.14,0.14,-2.47,-1.01
3,-1.24,0.04,-2.27,-1.01
4,-0.84,0.54,-2.37,-1.01
...,...,...,...,...
145,0.86,-0.06,1.43,1.09
146,0.46,-0.56,1.23,0.69
147,0.66,-0.06,1.43,0.79
148,0.36,0.34,1.63,1.09


## 11.2 applymap

In [192]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [195]:
member['month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]) )
member

,Name,Age,Email,Address,Birth,Year,month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [206]:
#연습하기 (apply,map)
# new_df의 연도월 데이터를 연도와 월을 정수로 분리하여 컬럼을 추가하세요.
new_df['연도'] = new_df['연도월'].apply(lambda str : int(str.split('년')[0]),)
new_df
new_df['월'] = new_df['연도월'].apply(lambda str : int(str.split('년')[1].replace('월','')),)
new_df

,지역,연도월,평당분양가,월,연도
0,서울,2013년12월,18189,12,2013
1,부산,2013년12월,8111,12,2013
2,대구,2013년12월,8080,12,2013
3,인천,2013년12월,10204,12,2013
4,광주,2013년12월,6098,12,2013
...,...,...,...,...,...
352,전북,2015년8월,6580,8,2015
353,전남,2015년8월,6289,8,2015
354,경북,2015년8월,7037,8,2015
355,경남,2015년8월,7665,8,2015


In [207]:
new_df.sample()

,지역,연도월,평당분양가,월,연도
178,세종,2014년10월,8527,10,2014


In [ ]:
#2차원 데이터 프레임의 apply 이용하는 방법

In [208]:
def year_month_append(df):
    return df

In [209]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
월              12
연도           2013
Name: 0, dtype: object

In [212]:
def year_month_append(df):
    '''
    df를 받아 연도와 월 추가 후 return
    '''
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [213]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
월              12
연도           2013
Name: 0, dtype: object

In [233]:
def split_date(df):
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [238]:
split_date(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분양가       18189
연도           2013
월              12
Name: 0, dtype: object

In [242]:
new_df = new_df.apply(split_date, axis=1)
new_df.head()

,지역,연도월,평당분양가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절. 일괄변경하기(결측치,특정값)
## 12.1 fillna (결측치 채우는 함수) : 결측치를 특정한 값으로 변경
- 대부분 결측치는 drop하거나, 평균값대체, 중위수대체, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [244]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                   columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [246]:
# 모든 결측치에 0 넣기
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [248]:
#결측치를 이전행의 값으로 대체
df.fillna(method='ffill') 

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [251]:
#결측치를 다음값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [256]:
#결측치를 열마다 특정값으로 대체
values = {'A':85,'B':80,'C':75,'D':90}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [261]:
#결측치는 열별 평균값으로 대체
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [263]:
#결측치는 열별 중위수로 대체 
df.fillna(value=df.median(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [266]:
# 2번째 결측치까지만 대체 (결측치 기준으로 2번째)
df.fillna(value=values, limit=2)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


In [3]:
import pandas as pd
import numpy as np

# 연습문제

#### 성미 풀이

In [57]:
# 연습문제
# fiilna / apply
'''
1  ) iris데이터를 불러와서 각 열마다 결측치 20개씩 할당
     결측치가 각 열마다 존재하는지 확인(info말고 함수)
2-1) 결측치는 각 열의 평균으로 대체 : fillna사용
2-2) 결측치는 종별, 열별 중위수로 대체 (groupby, pivot_table)로 대체 : fillna사용하지 말고 apply 사용
'''
#iris 데이터 불러오기
import seaborn as sns
iris = sns.load_dataset('iris')
#iris.head()

import random
print(random.sample(range(150),20))
for col in range(iris.shape[1]-1):
    iris.iloc[random.sample(range(150),20),col] = np.nan


[131, 81, 14, 142, 19, 4, 49, 78, 54, 116, 125, 48, 5, 40, 104, 121, 37, 95, 26, 9]


In [17]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  130 non-null    float64
 1   sepal_width   130 non-null    float64
 2   petal_length  130 non-null    float64
 3   petal_width   130 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [19]:
iris.groupby('species').count()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,45,45,45,43
versicolor,42,41,44,44
virginica,43,44,41,43


In [26]:
ir_t= iris

In [31]:
ir_t.fillna(value=ir_t.median(axis=0, numeric_only=True))

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,4.2,0.2,setosa
1,4.9,3.0,1.4,1.3,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.0,1.5,0.2,setosa
4,5.0,3.6,1.4,1.3,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,5.7,3.0,4.2,1.3,virginica
148,6.2,3.4,5.4,2.3,virginica


In [32]:
#결측치는 종별, 열별 중위수(groupby, pivot_table)로 대체 : fillna사용하지 말고 apply 사용

In [ ]:
# 

#### 선생님 풀이

In [34]:
import random 
import numpy as np
import pandas as pd
import seaborn as sns
iris = sns.load_dataset('iris')
rowlen, collen = iris.shape
rowlen, collen

(150, 5)

In [35]:
#결측치 할당 후 확인
print('임의의 20개 index 추출 :',random.sample(range(rowlen), 20))

임의의 20개 index 추출 : [122, 51, 117, 87, 8, 92, 39, 53, 58, 44, 32, 60, 102, 81, 104, 103, 98, 114, 45, 1]


In [36]:
#결측치 할당
for col in range(collen-1):
    iris.iloc[random.sample(range(rowlen),20),col] = np.nan

In [37]:
#결측치 확인_info()
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  130 non-null    float64
 1   sepal_width   130 non-null    float64
 2   petal_length  130 non-null    float64
 3   petal_width   130 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [42]:
#결측치 확인 1)iris.isna() :결측치 여부 / False =0, True=1 
#결측치 갯수 확인
iris.isna().sum() 

sepal_length    20
sepal_width     20
petal_length    20
petal_width     20
species          0
dtype: int64

In [46]:
# 결측치 확인 2)isnull()
# isnull = isna
iris.isnull()

,sepal_length,sepal_width,petal_length,petal_width,species
0,False,True,False,False,False
1,False,False,True,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,True,False,False,False
...,...,...,...,...,...
145,True,False,False,False,False
146,False,False,False,True,False
147,False,False,False,False,False
148,False,False,False,False,False


In [56]:
# (2-1단계) 결측치는 각 열의 중위수로 대체한다 : fillna사용
iris.median(numeric_only=True)
iris.iloc[:,:-1].median()
iris.fillna(value=iris.iloc[:,:-1].median())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.0,1.4,0.2,setosa
1,4.9,3.0,4.2,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.0,1.4,0.2,setosa
...,...,...,...,...,...
145,5.8,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.3,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [55]:
iris.fillna(value=iris.median(axis=0 , numeric_only=True))

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.0,1.4,0.2,setosa
1,4.9,3.0,4.2,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.0,1.4,0.2,setosa
...,...,...,...,...,...
145,5.8,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.3,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [ ]:
ir_t.fillna(value=ir_t.median(axis=0, numeric_only=True))

In [67]:
# 결측치가 어디에 있는지 확인하기 (하나라도 있는 행 출력)
iris[iris['sepal_length'].isna() | iris['sepal_width'].isna() | 
     iris['petal_length'].isna() | iris['petal_width'].isna()]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,NaN,setosa
4,NaN,3.6,1.4,0.2,setosa
5,NaN,3.9,1.7,NaN,setosa
6,NaN,3.4,1.4,0.3,setosa
11,4.8,3.4,1.6,NaN,setosa
...,...,...,...,...,...
140,6.7,3.1,NaN,2.4,virginica
144,NaN,3.3,5.7,NaN,virginica
146,6.3,NaN,NaN,1.9,virginica
148,6.2,NaN,NaN,2.3,virginica


In [68]:
#결측치를 열별 평균으로 대체 
iris.fillna(value=iris.iloc[:,:-1].mean())

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.10,3.500000,1.400000,1.192308,setosa
1,4.90,3.000000,1.400000,0.200000,setosa
2,4.70,3.200000,1.300000,0.200000,setosa
3,4.60,3.100000,1.500000,0.200000,setosa
4,5.84,3.600000,1.400000,0.200000,setosa
...,...,...,...,...,...
145,6.70,3.000000,5.200000,2.300000,virginica
146,6.30,3.076923,3.717692,1.900000,virginica
147,6.50,3.000000,5.200000,2.000000,virginica
148,6.20,3.076923,3.717692,2.300000,virginica


In [74]:
# iris.isna().sum()
# iris.isnull().sum()

In [77]:
# 결측치를 종별 열별 평균(groupby,pivot_table)로 대체한다.
iris_mean = iris.groupby(by='species').mean()
iris_mean

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.020930,3.429787,1.469565,0.250000
versicolor,5.936364,2.784091,4.307317,1.327907
virginica,6.560465,2.982051,5.560465,2.020930


In [106]:
# 결측치를 종별, 열별 평균값으로 할당하는 함수 만들기
iris.iloc[4]

sepal_length       NaN
sepal_width        3.6
petal_length       1.4
petal_width        0.2
species         setosa
Name: 4, dtype: object

In [91]:
iris_mean, #iris_mean.loc['setosa','sepal_length']

(            sepal_length  sepal_width  petal_length  petal_width
 species                                                         
 setosa          5.020930     3.429787      1.469565     0.250000
 versicolor      5.936364     2.784091      4.307317     1.327907
 virginica       6.560465     2.982051      5.560465     2.020930,
 5.02093023255814)

In [109]:
#iris 한 행을 매개변수로 받아 결측치가 있는 경우 종별 열별 중위수로 대체하는 함수
#결측치인지 묻고, species인지 물어보고 컬럼네임 확인 후 값 변경

def iris_fillna(irisrow):
    irisrow = irisrow.copy()
    for col, value in irisrow.items():
        if isinstance(value, (int, float, np.number)) and np.isnan(value):  #결측치면 true 
#             print(irisrow[col], '이 결측치') #값을 수정할 때  대괄호[]
            irisrow[col] = iris_mean.loc[irisrow.species, col]
    return irisrow

In [115]:
iris_fillna(iris.iloc[16])

sepal_length         5.4
sepal_width          3.9
petal_length    1.469565
petal_width          0.4
species           setosa
Name: 16, dtype: object

In [120]:
#apply 적용
iris = iris.apply(iris_fillna, axis=1)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.10000,3.5,1.4,0.25,setosa
1,4.90000,3.0,1.4,0.20,setosa
2,4.70000,3.2,1.3,0.20,setosa
3,4.60000,3.1,1.5,0.20,setosa
4,5.02093,3.6,1.4,0.20,setosa


In [123]:
iris.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [78]:
#iris_mean이 2차원이라 fillna가 안 됨
iris.fillna(iris_mean)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,NaN,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,NaN,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,NaN,NaN,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,NaN,NaN,2.3,virginica


## 12.2 replace(to_value,new_value,inplace=T/F)
- to_value를 new_value로 변경
- to_value에 <font color='red'>**정규표현식 이용** </font>
- 데이터 프레임, 시리즈 적용

In [134]:
s = pd.Series([ 1, 2, 3, 4, np.nan]) #column 이름 안 쓰면 자동으로 index는 0,1,2,3으로 표현
s.replace(np.nan,0)
s.replace([1,4],99) #0이나 1을 99로 변경 / 0번방이 아님
s.replace([1,2], [100,101]) #0은 100으로, 2는 101으로 변경

0    100.0
1    101.0
2      3.0
3      4.0
4      NaN
dtype: float64

In [142]:
df = pd.DataFrame({'A': ['bat','foo','balt'],
                   'B' : ['abc', 'gee','baby']})
df.replace(r'ba.+', 'XX', regex=True)   #ba로 들어가는 단어를 XX로 바꾸기 
# regex에서 true는 정규표현식으로 찾아서 하고 false는 같은 단어만 찾아서 변경

,A,B
0,XX,abc
1,foo,gee
2,XX,XX


## 12.3 mask(조건), where(조건)
- numpy에서도 사용 가능 
___
- where(조건) : 조건이 True인 요소만 출력하고, 그 외 요소는 결측치로 출력
- mask(조건) : 조건이 False인 요소만 출력하고, 그 외 요소는 결측치로 출력 \
※ mask와 where은 반대
___

In [145]:
s = pd.Series(range(10,20))
s

0    10
1    11
2    12
3    13
4    14
5    15
6    16
7    17
8    18
9    19
dtype: int64

In [147]:
s.where(s>15)

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6    16.0
7    17.0
8    18.0
9    19.0
dtype: float64

In [148]:
s.mask(s>15)

0    10.0
1    11.0
2    12.0
3    13.0
4    14.0
5    15.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

## 12.4 dropna 
- 결측치가 있는 데이터를 누락(행,열)

In [150]:
df = pd.DataFrame({'name':[ '홍','김','이', np.nan],
                   'age':[np.nan, 20,30,np.nan],
                   'born': ['서울','부산',np.nan,np.nan]})
df

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN
3,NaN,NaN,NaN


In [151]:
df.dropna(how='all',) #all 모두 결측치인 행만 삭제

,name,age,born
0,홍,NaN,서울
1,김,20.0,부산
2,이,30.0,NaN


In [154]:
df.dropna() #결측치가 하나라도 있는 행 삭제

,name,age,born
1,김,20.0,부산


In [155]:
df.dropna(axis=1) #결측치가 하나라도 있는 열 삭제

""
0
1
2
3


In [156]:
# df.dropna(axis=0) = df.dropna()과 동일 / axis=0은 기본값

,name,age,born
1,김,20.0,부산


In [157]:
#name과 age이 np.nan인 것만 지우기 / born의 결측치는 냅두기 / 결측치는 folat
df.dropna(subset=['name','age'])

,name,age,born
1,김,20.0,부산
2,이,30.0,NaN


# 13절. 시리즈
- 숫자로 되어있으면 sort가 됨.

In [159]:
s = pd.Series([1000, 7000, 5000 ],
              index=['mango','apple','banana'])
s

mango     1000
apple     7000
banana    5000
dtype: int64

In [165]:
s['mango'] = 2800 #데이터 수정

In [166]:
s['orange'] = 4600 #데이터 추가

In [167]:
s

mango     2800
apple     7000
banana    5000
orange    4600
dtype: int64

In [173]:
s['mango':'banana'] #슬라이싱 가능 [from:to] from부터 to까지 (to포함)

mango     2800
apple     7000
banana    5000
dtype: int64

In [175]:
s[['mango','banana']]

mango     2800
banana    5000
dtype: int64

In [177]:
s.sort_index()

apple     7000
banana    5000
mango     2800
orange    4600
dtype: int64

In [181]:
s.sort_values(ascending=False) #기본이 오름차순 정렬

apple     7000
banana    5000
orange    4600
mango     2800
dtype: int64

In [192]:
s.to_frame().T

,mango,apple,banana,orange
0,2800,7000,5000,4600


# 14절 연습문제

In [204]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [276]:
#1. iris 데이터에서 처음 다섯개 행만 출력
iris.head()
iris.loc[:5]
iris.iloc[:5]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [283]:
#2. iris 데이터를 독립변수 x와 종속변수y로 나누세요.
X = iris.iloc[:,:-1]
y = iris.iloc[:,-1:]

X = iris.loc[:,'sepal_length':'petal_width']
y = iris.loc[:,'species':]


# from sklearn import datasets
# iris_sk = datasets.load_iris()
# X = pd.DataFrame(iris_sk.data,columns=iris_sk.feature_names)

# # Y = pd.Series(iris_sk.columns=='species')
# #Y 다시 구해보기 

# print('독립변수 X = ',X.head(),sep='\n')
# print('종속변수 Y =',Y,sep='\n')

SyntaxError: invalid syntax (1608612722.py, line 4)

In [287]:
# 3. iris 데이터에서 처음 50개행을 빼내서 temp 변수에 저장
temp = iris.head(50)
temp1 = iris.iloc[:50]
temp2 = iris.loc[:49]
temp3 = iris[:50]
all(temp==temp1), all(temp2==temp3), all(temp==temp2), temp.shape


(True, True, True, (50, 5))

In [288]:
def my_all(*answers):
    if len(answers)>0:
        return all(all(answers[0]==answer) for answer in answers)
    return False

In [290]:
my_all(temp, temp1,temp2,temp3)

True

In [226]:
# 4. 3번에서 선택한 데이터 프레임의 요약정보를 출력하세요. 모든열에 대한 요약정보가 출력이어야 함
temp.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species
count,50.00000,50.000000,50.000000,50.000000,50
unique,NaN,NaN,NaN,NaN,1
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.00600,3.428000,1.462000,0.246000,NaN
std,0.35249,0.379064,0.173664,0.105386,NaN
min,4.30000,2.300000,1.000000,0.100000,NaN
25%,4.80000,3.200000,1.400000,0.200000,NaN
50%,5.00000,3.400000,1.500000,0.200000,NaN
75%,5.20000,3.675000,1.575000,0.300000,NaN


In [268]:
#5. versicolor종의 데이터만 iris_versicolor 변수에 저장
# iris_versicolor = iris.loc[iris.species.str.contains('versicolor')]
# iris_versicolor = iris.loc[iris['species']=='versicolor']
# iris_versicolor = iris.loc[iris.species=='versicolor']
# iris_versicolor = iris[iris.species=='versicolor']
# iris_versicolor


iris_versicolor = iris[iris.species == 'versicolor']
iris_versicolor

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
53,5.5,2.3,4.0,1.3,versicolor
54,6.5,2.8,4.6,1.5,versicolor
55,5.7,2.8,4.5,1.3,versicolor
56,6.3,3.3,4.7,1.6,versicolor
57,4.9,2.4,3.3,1.0,versicolor
58,6.6,2.9,4.6,1.3,versicolor
59,5.2,2.7,3.9,1.4,versicolor


In [291]:
# 6 2번의 X와 Y변수를 합해서 iris_df 데이터 프레임을 만들어라.
# pd.concat([X,Y,],axis=1)
#2번의 값이 데이터프레임이 아니라서 병합이 안 되나..
import pandas as pd
pd.concat([X,y], axis=1)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [293]:
#7. iris 데이터의 각 열 평균값을 출력
import numpy as np
iris.iloc[:,:-1].mean(axis=0)  # axis=0 기본값 생략 가능
iris.iloc[:,:-1].apply(np.mean, axis=0) #비추 

# am = iris.iloc[:,:-1]
# am.std(axis='index')

sepal_length    5.843333
sepal_width     3.057333
petal_length    3.758000
petal_width     1.199333
dtype: float64

In [295]:
#8. iris 데이터의 각 열들 사이의 상관계수를 출력
iris.corr(numeric_only=True)
iris.iloc[:,:-1].corr()


,sepal_length,sepal_width,petal_length,petal_width
sepal_length,1.000000,-0.117570,0.871754,0.817941
sepal_width,-0.117570,1.000000,-0.428440,-0.366126
petal_length,0.871754,-0.428440,1.000000,0.962865
petal_width,0.817941,-0.366126,0.962865,1.000000


In [297]:
#9. iris데이터에서의 각 요소들과 변수별 평균과의 차이를 출력하세요.
iris_mean = iris.iloc[:,:-1].mean() # iris.mean(numeric_only=True)동일
iris_mean
iris.iloc[:,:-1].apply(lambda x : x-iris_mean, axis=1)

,sepal_length,sepal_width,petal_length,petal_width
0,-0.743333,0.442667,-2.358,-0.999333
1,-0.943333,-0.057333,-2.358,-0.999333
2,-1.143333,0.142667,-2.458,-0.999333
3,-1.243333,0.042667,-2.258,-0.999333
4,-0.843333,0.542667,-2.358,-0.999333
...,...,...,...,...
145,0.856667,-0.057333,1.442,1.100667
146,0.456667,-0.557333,1.242,0.700667
147,0.656667,-0.057333,1.442,0.800667
148,0.356667,0.342667,1.642,1.100667


In [255]:
#10. iris 데이터의 종별 평균을 출려
iris.pivot_table(index='species',
                 aggfunc='mean')

# iris.pivot_table(index='species', 
#         # values=['sepal_length','sepal_width','petal_length','petal_width'],
#         # aggfunc='mean'
#         )
# iris.groupby(['species']).mean()

,petal_length,petal_width,sepal_length,sepal_width
species,,,,
setosa,1.462,0.246,5.006,3.428
versicolor,4.260,1.326,5.936,2.770
virginica,5.552,2.026,6.588,2.974


# 여기 푸는중

In [298]:
# 11. iris 데이터에서 각 요소들의 종별 변수의 평균과의 차이를 출력. 각 종별로 3개씩 출력 
iris_mean_by_species = iris.groupby(iris.species).mean()
iris_mean_by_species 

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [307]:
iris_grouped_sample.apply(lambda row:rwo-iris_mean_by_species[row.species])(iris_grouped_sample,iloc[0])

NameError: name 'rwo' is not defined

In [ ]:
iris.loc[0] - iris_mean_by_species.loc[iris.loc[0].species]

In [299]:
#series = iris.loc[0] 과 동일 
result = iris.apply(lambda series : series-iris_mean_by_species.loc[series.species], axis=1)
# result.iloc[list(range(3))+list(range(50,53))+list(range(100,103))]
result.iloc[[0,1,2,50,51,52,100,101,102]]
result.groupby(iris.species).head(3)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN


In [300]:
result.groupby(iris.species).head(3)

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.062,-0.046,0.094,0.072,NaN
1,-0.062,-0.046,-0.106,-0.428,NaN
2,-0.162,-0.046,-0.306,-0.228,NaN
50,0.440,0.074,1.064,0.430,NaN
51,0.240,0.174,0.464,0.430,NaN
52,0.640,0.174,0.964,0.330,NaN
100,0.448,0.474,-0.288,0.326,NaN
101,-0.452,-0.126,-0.788,-0.274,NaN
102,0.348,0.074,0.512,0.026,NaN


In [301]:
# 11 각 요소들과 종별 변수의 평균과의 차이를 각 종별로 3개씩 출력(방법2)
iris_grouped_sample = iris.groupby('species').take([0,1,2])
iris_grouped_sample

sepal_length  sepal_width  petal_length  petal_width
species                                                             
setosa     1             4.9          3.0           1.4          0.2
           2             4.7          3.2           1.3          0.2
           3             4.6          3.1           1.5          0.2
versicolor 51            6.4          3.2           4.5          1.5
           52            6.9          3.1           4.9          1.5
           53            5.5          2.3           4.0          1.3
virginica  101           5.8          2.7           5.1          1.9
           102           7.1          3.0           5.9          2.1
           103           6.3          2.9           5.6          1.8

In [302]:
iris_grouped_sample = iris.groupby('species').head(3)
iris_grouped_sample



,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica


KeyError: "['None'] not found in axis"

In [167]:
member['Birth'].str.split('-')[3][1] 

'05'